In [1]:
using FileIO
using GLMakie
GLMakie.activate!()

In [2]:
# Load Object Files
tetra = load("./data/models/4_tetrahedron.obj")
hexa = load("./data/models/6_hexahedron.obj")
octa = load("./data/models/8_octahedron.obj")
dodeca = load("./data/models/12_dodecahedron.obj")
icosa = load("./data/models/20_icosahedron.obj")
""

""

In [21]:
scene = Scene()
cam3d!(scene)

tm = mesh!(scene, tetra, color=:blue)
tw = wireframe!(scene, tetra)
translate!(tm, 5, 0, 0)
translate!(tw, 5, 0, 0)

hm = mesh!(scene, hexa, color=:green)
hw = wireframe!(scene, hexa)
translate!(hm, 0, 5, 0)
translate!(hw, 0, 5, 0)

om = mesh!(scene, octa, color=:yellow)
ow = wireframe!(scene, octa)
translate!(om, -5, 0, 0)
translate!(ow, -5, 0, 0)

dm = mesh!(scene, dodeca, color=:orange)
dw = wireframe!(scene, dodeca)
translate!(dm, 0, -5, 0)
translate!(dw, 0, -5, 0)

im = mesh!(scene, icosa, color=:red)
iw = wireframe!(scene, icosa)
scene

In [4]:
fig = Figure()
ax = Axis(fig[1,1], title = "y² = x³ + ax + b")


# Slider Objects 
sa = Slider(fig[2,1][1,1], range = -5:0.01:5, startvalue = 1)
sb = Slider(fig[2,1][1,2], range = -5:0.01:5, startvalue = 1)
la = lift(a -> string("a = ", a), sa.value)
lb = lift(b -> string("b = ", b), sb.value)
Label(fig[2,1][2, 1], la, tellwidth = false)
Label(fig[2,1][2, 2], lb, tellwidth = false)

data = lift(sa.value, sb.value) do a, b
    function f(x, y)
        x^3 + a*x + b - y^2
    end
    return f
end

# X and Y sample range
xs = range(-8, 8, length=100)
ys = range(-6, 8, length=100)

# Plot Contour
contour!(ax, xs, ys, data; levels=[0])

# Adjust Axis limits
on(data) do val 
    autolimits!(ax) 
end

display(fig)

GLMakie.Screen(...)

In [2]:
function parametric_grid(us, vs, r)
    n,m = length(us), length(vs)
    xs, ys, zs = zeros(n,m), zeros(n,m), zeros(n,m)
    for (i, uᵢ) in pairs(us)
        for (j, vⱼ) in pairs(vs)
            x,y,z = r(uᵢ, vⱼ)
            xs[i,j] = x
            ys[i,j] = y
            zs[i,j] = z
        end
    end
    (xs, ys, zs)
end

parametric_grid (generic function with 1 method)

In [3]:
fig = Figure()

# ax = Axis(fig[1,1], title = "Interactive Torus")
ax = Axis3(fig[1,1], title = "Interactive Torus")
xlims!(ax,-5,5)
ylims!(ax,-5,5)
zlims!(ax,-1,1)

# Slider Objects 
sr1 = Slider(fig[2,1][1,1], range = 2:0.1:4, startvalue = 2)
sr2 = Slider(fig[2,1][1,2], range = 0.5:0.1:1.5, startvalue = 0.5)
lr1 = lift(r1 -> string("R₁ = ", r1), sr1.value)
lr2 = lift(r2 -> string("R₂ = ", r2), sr2.value)
Label(fig[2,1][2, 1], lr1, tellwidth = false)
Label(fig[2,1][2, 2], lr2, tellwidth = false)

# inital plot 
r1 = 2; r2 = 1/2
us = vs = range(0, 2pi, length=25)
r(u,v) = ((r1 + r2*cos(v))*cos(u), (r1 + r2*cos(v))*sin(u), r2*sin(v))
xs, ys, zs = parametric_grid(us, vs, r)
surface!(ax, xs, ys, zs)

# Parametric Equation updata Observable
fr = lift(sr1.value, sr2.value) do R₁, R₂
    function f(u, v)
        ((R₁ + R₂*cos(v))*cos(u), (R₁ + R₂*cos(v))*sin(u), R₂*sin(v))
    end
    return f
end

# X Y Z data update Observable
result = lift(fr) do R
    xs, ys, zs = parametric_grid(us, vs, R)
    return (xs, ys, zs)
end 

# add new surface on obsevable
on(result) do (xs, ys, zs)
    while !isempty(ax.scene.plots)
        delete!(ax.scene, ax.scene.plots[end])
    end
    surface!(ax, xs, ys, zs)
    # autolimits!(ax)
end

display(fig)

GLMakie.Screen(...)